# Mask Detector  

In [5]:
import cv2
import os

In [6]:
features = []
target = []
for i in ["with_mask","without_mask"]:
    collection_image_names = os.listdir("D:/data/" + str(i))
    for j in collection_image_names:
        img = cv2.imread("D:/data/"+  str(i) + "/" + j)
        try:
            img = cv2.resize(img,(32,32))
        except:
            pass
        else:
            features.append(img)
            if i=="WithMask":
                target.append(0)
            else:
                target.append(1)       
    print("Loaded Folder ",i)

Loaded Folder  with_mask
Loaded Folder  without_mask


In [7]:
import numpy as np
features = np.array(features)
target = np.array(target)

In [8]:
def preprocessing(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = image/225
    return image

In [9]:
features = np.array(list(map(preprocessing,features)))

In [10]:

from sklearn.model_selection import train_test_split
train_features,test_features,train_target,test_target = train_test_split(features,target,test_size=0.2)

In [11]:
train_features.shape

(6042, 32, 32)

In [12]:
train_features = train_features.reshape(6042,32,32,1)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
dataGen = ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)
dataGen.fit(train_features)
batches = dataGen.flow(train_features,train_target,batch_size=20)
images,labels = next(batches)

In [15]:
images.shape

(20, 32, 32, 1)

In [16]:
images = images.reshape(20,32,32)

In [17]:
from tensorflow.keras.utils import to_categorical
train_target = to_categorical(train_target)
train_target.shape

(6042, 2)

# Specifying Architecture

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,MaxPooling2D,Conv2D,Flatten,Dropout

In [19]:
model = Sequential()
model.add(Conv2D(100, (3,3), activation='relu', input_shape = (32,32,1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(30,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))

# Step 2: Compling the model

In [20]:
from tensorflow.keras.optimizers import Adam
model.compile(Adam(learning_rate=0.001),loss = "categorical_crossentropy",metrics=["accuracy"])

In [ ]:

model.fit(train_features,train_target,epochs=20,validation_split = 0.2)

Epoch 1/20
152/152 [==============================] - 8s 48ms/step - loss: 0.0058 - accuracy: 0.9988 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
152/152 [==============================] - 7s 47ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
152/152 [==============================] - 7s 47ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
152/152 [==============================] - 7s 46ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
152/152 [==============================] - 7s 47ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
152/152 [==============================] - 7s 47ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/20
152/152 [==============================] - 8s 53ms/step - loss: 0.0000e+00 - accuracy: 

# Step 3: Train the Model

In [3]:

location = "C:/Users/Nidhish/Downloads/AI/"

In [4]:

ModelInJson = model.to_json()
abc = open(location + "model_mask_detector2.json","w")
abc.write(ModelInJson)

model.save_weights(location + "model_mask_detector2_weights.h5")

NameError: name 'model' is not defined

In [28]:
from tensorflow.keras.models import model_from_json
abc = open(location + "model_mask_detector2.json","r")
loaded_data = abc.read()
loaded_model = model_from_json(loaded_data)
loaded_model.load_weights(location + "model_mask_detector2_weights.h5")

# Step 4: Testing Model

In [1]:
def getClassName(classNo):
    if classNo == 0:    return "YES"
    elif classNo == 1:  return "NO"

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np 

In [ ]:
capt=cv2.VideoCapture(0)
capt.set(3,640)
capt.set(4,480)
capt.set(10,180)

In [ ]:
def preprocessing(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = image/225
    return image

In [ ]:
while True:
    message,image=capt.read()
    imagearr=np.asarray(image)
    imagearr=cv2.resize(imagearr,(32,32))
    imagearr=preprocessing(imagearr)
    imagearr=imagearr.reshape(1,32,32,1)
    predictions=loaded_model.predict(imagearr)
    NeuronIndex=loaded_model.predict_classes(imagearr)
    cv2.putText(image,"Class: ",(20,35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
    cv2.putText(image,"Probablity: ",(20,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
    ProbablityValue = np.amax(predictions)
    if ProbablityValue > 0.25:
        cv2.putText(image,getClassname(NeuronIndex),(120,35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
        cv2.putText(image,str(int(ProbablityValue*100)) + "%",(200,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
    cv2.imshow("Mask Detector",cv2.resize(image,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capt.release()
cv2.destroyAllWindows()

In [2]:

from tensorflow.keras.models import load_model
import cv2
import numpy as np

cv2.ocl.setUseOpenCL(False)

capt = cv2.VideoCapture(0)


while True:
    msg,image = capt.read()
    face_cascade = cv2.CascadeClassifier("C:/Users/Nidhish/Downloads/AI/" + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.3,minNeighbors = 5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(image,(x,y-50),(x+w, y+h+10), (255,0,0),2)
        face_image = gray[y:y+h, x:x+w]
        resize = cv2.resize(face_image,(32,32))
        normalized = resize/225.0
        reshaped = np.reshape(normalized,(1,32,32,1))
        prediction = loaded_model.predict(reshaped)
        neuron_index = loaded_model.predict_classes(reshaped)
        
        cv2.putText(image,"Mask: ",(20,35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
        cv2.putText(image,"Probablity: ",(20,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
        ProbabilityValue = np.amax(prediction)
        if ProbabilityValue> 0.25:
            cv2.putText(image,getClassName(neuron_index),(120,35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
            cv2.putText(image,str(int(ProbabilityValue*100)) + "%",(200,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
    cv2.imshow('MASK DETECTOR', cv2.resize(image,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-q3d_8t8e\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'
